In [1]:
import pandas as pd
import wolf
from tasks import clumps_prep_task

In [2]:
clumps_localization_task = wolf.localization.BatchLocalDisk()

localization_results = clumps_localization_task.run(
    maf = "/home/adunford/data/MC3-14k-merge_v1.variant_classification.maf",
    genome_2bit = "gs://sa-clumps2-ref/dat/hg19.2bit",
    fasta = "gs://sa-clumps2-ref/dat/UP000005640_9606.fasta.gz",
    gpmaps = "gs://sa-clumps2-ref/dat/genomeProteomeMaps.txt",
    #prot2pdb_chunks = "gs://sa-clumps2-ref/dat/huniprot/huniprot2pdb.run18_chunks",
    pdb_dir = "gs://sa-clumps2-ref/dat/pdbs/ftp.wwpdb.org/pub/pdb/data/structures/divided/pdb",
    coverage_track = "gs://sa-clumps2-ref/dat/cov/WEx_cov.fwb",
    coverage_track_index = "gs://sa-clumps2-ref/dat/cov/WEx_cov.fwi",
    cancer_genes = "gs://sa-clumps2-ref/dat/allCancerGenes.txt",
    uniprot_map = "/home/adunford/data/huniprot2pdb.run18.filt.txt.gz"
)



[20230922-17:56:14] [prefect] Starting Slurm controller ...
[20230922-17:56:14] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20230922-17:56:14] [prefect] Started Slurm controller.
[20230922-17:56:14] [prefect] Workflow results disk low on space (30 GB remaining)
[20230922-17:56:14] [prefect] Hashing file /home/adunford/data/MC3-14k-merge_v1.variant_classification.maf; 100/344 MiB completed
[20230922-17:56:14] [prefect] Hashing file /home/adunford/data/MC3-14k-merge_v1.variant_classification.maf; 200/344 MiB completed
[20230922-17:56:14] [prefect] Hashing file /home/adunford/data/MC3-14k-merge_v1.variant_classification.maf; 300/344 MiB completed
[20230922-17:56:16] [prefect] Hashing directory gs://sa-clumps2-ref/dat/pdbs/ftp.wwpdb.org/pub/pdb/data/structures/divided/pdb. This may take some time.
[20230922-17:56:23] [prefect] Job avoidance disabled for this task; overwriting output.
[20230922-17:56:23] [prefect] Localizing inputs...
[20230922-17:56:30] [prefect] 

In [3]:
localization_results

<WolfTaskResults: State: COMPLETED
                  Inputs: {'pdb_dir': '<array>', 'coverage_track': '<array>', 'coverage_track_index': '<array>', 'uniprot_map': '<array>', 'cancer_genes': '<array>', 'genome_2bit': '<array>', 'fasta': '<array>', 'maf': '<array>', 'gpmaps': '<array>'}
                  Outputs: {cancer_genes, coverage_track, coverage_track_index, fasta, genome_2bit, gpmaps, maf, pdb_dir, uniprot_map}>

In [4]:
clumps_prep_results = clumps_prep_task(
    inputs = dict(
        inMaf = localization_results['maf'],
        genome_2bit = localization_results['genome_2bit'],
        #gpmaps = localization_results['gpmaps'],
        fasta = localization_results['fasta'],
        scatterWidth =  350,
        huniprot2pdb = localization_results['uniprot_map']
    ))


[20230922-17:58:02] [prefect] Starting Slurm controller ...
[20230922-17:58:02] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20230922-17:58:02] [prefect] Started Slurm controller.
[20230922-17:58:02] [prefect] Workflow results disk low on space (32 GB remaining)
[20230922-17:58:03] [prefect] Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/clumps_prep_task__2023-09-22--17-58-03_0abueji_lnmtxla_c5qzcu114xoik/jobs/0'
[20230922-17:58:03] [prefect] Overwriting output and aborting job avoidance.
[20230922-17:58:03] [prefect] Localizing inputs...
[20230922-17:58:03] [prefect] Task staged in /mnt/nfs/workspace/clumps_prep_task__2023-09-22--17-58-03_0abueji_lnmtxla_c5qzcu114xoik
[20230922-17:58:03] [prefect] 1 job submitted.
[20230922-18:23:15] [prefect] Finished with status COMPLETED


In [5]:
clumps_prep_results

<WolfTaskResults: State: COMPLETED
                  Inputs: {'fasta': '<array>', 'genome_2bit': '<array>', 'huniprot2pdb': '<array>', 'inMaf': '<array>', 'scatterWidth': '<array>'}
                  Outputs: {mutations, sampleMutSpectra, sampleMutFreq, prot2pdbchunks}>

In [6]:
from tasks import clumps_run_task
clumps_run_task.script

"\n    ## making sure we're writing to local disk (not boot disk)\n    mkdir sw\n    cp -vr /sw/* sw\n    rm -rf /sw\n    ln -vs $PWD/sw /sw\n    #unpack mutations from clumps prep\n    #tar xzvf ${mutationsTarball} && mv splitByProtein /sw/dat/\n    cp -r ${mutationsTarball} /sw/dat/\n\n    #link 2bit, fasta locally a\n    ln ${genome_2bit} /sw/src/\n    ln ${fasta}  /sw/src/\n    \n    mkdir -p /sw/dat/ftp.wwpdb.org/pub/pdb/data/structures/divided/\n    ln ${pdb_dir} /sw/dat/ftp.wwpdb.org/pub/pdb/data/structures/divided/ #... think I need to do this so pdb is in expected location? \n    \n    #then this is done to parse through huniprot list\n    if [ ${lineId} = -1 ]; then\n        nlines=`zcat ${prot2pdb_chunks} | wc -l`\n        lines=$(seq 1 $nlines)\n    else\n        lines=${lineId}\n    fi\n    \n    START_DIR=$PWD\n    cd /sw/src\n    for line in $lines;     do\n        echo python clumps2.py ${setfile} ${timeout} ${nthreads} ${prot2pdb_chunks} $line ${ttype} ${sampler} ${sam

In [ ]:

clumps_results = clumps_run_task(                                                                                                                                                                               
                                                                                                                                                                                                                                    
    inputs = dict(
        mutationsTarball  = clumps_prep_results['mutations'],
        sampleMutFreq = clumps_prep_results['sampleMutFreq'],
        sampleMutSpectra = clumps_prep_results['sampleMutSpectra'],
        setfile = '/home/adunford/data/14k.run4',
        prot2pdb_chunks = clumps_prep_results['prot2pdbchunks'],
        pdb_dir = localization_results['pdb_dir'],
        coverage_track = localization_results['coverage_track'],
        coverage_track_index = localization_results['coverage_track_index'], # not actually used as an input; just needs to be localized alongside coverage_track                                                                                                                                                   
        genome_2bit = localization_results['genome_2bit'],
        fasta = localization_results['fasta'],
        gpmaps = localization_results['gpmaps'],
        sampler = "MutspecCoverageSampler",
        lineId = -1,
        nthreads = 8,
        ttype = "PanCan",
        timeout = 7200

    )
)

[20230922-18:23:15] [prefect] Starting Slurm controller ...
[20230922-18:23:15] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20230922-18:23:15] [prefect] Started Slurm controller.
[20230922-18:23:15] [prefect] Workflow results disk low on space (31 GB remaining)
[20230922-18:23:16] [prefect] Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/clumps_run_task__2023-09-22--18-23-16_lv0axny_lnmtxla_x5udy0irircts/jobs/0'
[20230922-18:23:16] [prefect] Overwriting output and aborting job avoidance.
[20230922-18:23:16] [prefect] Localizing inputs...
[20230922-18:23:42] [prefect] Task staged in /mnt/nfs/workspace/clumps_run_task__2023-09-22--18-23-16_lv0axny_lnmtxla_x5udy0irircts
[20230922-18:23:42] [prefect] 350 jobs submitted.


In [13]:
clumps_results

<WolfTaskResults: States: FAILED: 192, COMPLETED: 152, OUT_OF_ME+: 6
                  Inputs: {'coverage_track': '<array>', 'coverage_track_index': '<array>', 'fasta': '<array>', 'genome_2bit': '<array>', 'gpmaps': '<array>', 'lineId': '<array>', 'max_perms': '<array>', 'mutationsTarball': '<array>', 'nthreads': '<array>', 'pdb_dir': '<array>', 'prot2pdb_chunks': '<array>', 'sampleMutFreq': '<array>', 'sampleMutSpectra': '<array>', 'sampler': '<array>', 'setfile': '<array>', 'timeout': '<array>', 'ttype': '<array>'}
                  Outputs: {run_outdir}>

In [16]:
%load_ext autoreload
%autoreload 2
from tasks import clumps_postprocess_task
clumps_postprocess_task.script

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'\n        \n        \n        for file in `cat ${clumpsScanOut} | grep -v nan`\n        do\n            tar xzvf $file\n        done\n        mv sw/res/clumps /sw/res/\n        \n        \n        mv ${mutationsSplitByProtein} /sw/dat/\n        python /sw/src/clumps_postprocess.py ${setfile}  /sw/res/clumps ${cancerGeneList} ${huniprot2pdb} ${ttype}\n\n\n    '

In [17]:
clumps_post_results = clumps_postprocess_task(
    inputs = dict(
        mutationsSplitByProtein =clumps_prep_results['mutations'],
        clumpsCandidatesOut = [clumps_results['run_outdir']], # this is an array of directories, which in turn contain multiple files                                                                                                                                                                                                                                                             
        huniprot2pdb = localization_results['uniprot_map'],
        clumpsScanOut = [clumps_results['run_outdir']],
        cancerGeneList = localization_results['cancer_genes'],
        setfile = '/home/adunford/data/14k.run4',
        ttype = "PanCan"
    )
)

[20230925-17:12:31] [prefect] Starting Slurm controller ...
[20230925-17:12:31] [prefect] Waiting up to 60 seconds for Slurm controller to start ...
[20230925-17:12:32] [prefect] Started Slurm controller.
[20230925-17:12:32] [prefect] Workflow results disk low on space (28 GB remaining)
[20230925-17:12:32] [prefect] Cannot recover preexisting task outputs: [Errno 2] No such file or directory: '/mnt/nfs/workspace/clumps_postprocess_task__2023-09-25--17-12-32_cnunvzq_lnmtxla_a0w02qtgjt34k/jobs/0'
[20230925-17:12:32] [prefect] Overwriting output and aborting job avoidance.
[20230925-17:12:32] [prefect] Localizing inputs...
[20230925-17:12:37] [prefect] Task staged in /mnt/nfs/workspace/clumps_postprocess_task__2023-09-25--17-12-32_cnunvzq_lnmtxla_a0w02qtgjt34k
[20230925-17:12:37] [prefect] 1 job submitted.
[20230925-17:17:09] [prefect] Finished with status COMPLETED


In [ ]:
clumps_postprocess_task.docker

In [ ]:
clumps_prep_results['mutations']